# Poor hint Migration 
This notebook demonstrates how an LLM performs when presented with a Spring Security migration issue without an example.

Installing pre requisites and configuring Kai with `Llama-3.1-8B-Instruct`

In [ ]:
%pip uninstall kai -y
%pip install --no-cache-dir git+https://github.com/konveyor/kai.git@main
%pip install python-dotenv

In [30]:
from IPython.display import display, Markdown
from kai.llm_interfacing.model_provider import ModelProvider
from kai.kai_config import KaiConfigModels, SupportedModelProviders
from dotenv import load_dotenv
import os
load_dotenv(override=True) 
# Initialize the model provider using Llama-3.1-8B-Instruc via OpenAI
model = ModelProvider.from_config(KaiConfigModels(
    provider=SupportedModelProviders.CHAT_OPENAI,
    args={"model": "meta-llama/Llama-3.1-8B-Instruct",
        "base_url": "https://llama-3-1-8b-instruct-maas-apicast-production.apps.prod.rhoai.rh-aiservices-bu.com:443/v1",
        "api_key": os.getenv("OPENAI_API_KEY"),},
))

# Async rendering function for displaying the response
async def rendered_llm_call(prompt: str):
    response = await model.ainvoke_llm(prompt)
    display(Markdown(response.content))
    return response



Below is the snippet we are trying to migrate from springboot2 to springboot3. This code uses the discouraged `configure(AuthenticationManagerBuilder)` method inside a `WebSecurityConfigurerAdapter`, both of which are deprecated in Spring Security 5.7+ and removed in Spring Boot 3.

Our goal is to assess how well the LLM refactors this configuration without a hint, and whether it can replace deprecated patterns with the right approach.

In [31]:
before_code = """\
import org.springframework.context.annotation.Bean;
import org.springframework.context.annotation.Configuration;
import org.springframework.security.config.annotation.authentication.builders.AuthenticationManagerBuilder;
import org.springframework.security.config.annotation.web.configuration.EnableWebSecurity;
import org.springframework.security.config.annotation.web.configuration.WebSecurityConfigurerAdapter;
import org.springframework.security.crypto.bcrypt.BCryptPasswordEncoder;
import org.springframework.security.crypto.password.PasswordEncoder;

@Configuration
@EnableWebSecurity
public class SecurityConfig extends WebSecurityConfigurerAdapter {
    @Override
    protected void configure(AuthenticationManagerBuilder auth) throws Exception {
        auth.inMemoryAuthentication()
            .withUser("user")
            .password(passwordEncoder().encode("password"))
            .roles("USER");
    }

    @Bean
    public PasswordEncoder passwordEncoder() {
        return new BCryptPasswordEncoder();
    }
}
"""

This rule flags the use of `configure(AuthenticationManagerBuilder)`. In Spring Boot 3, developers should instead use a `UserDetailsService` and `PasswordEncoder` bean for authentication configuration.

In [32]:
message=""""\

  description: configure(AuthenticationManagerBuilder) override is discouraged since Spring Security 5.7.
  message: |
    Overriding `configure(AuthenticationManagerBuilder)` is **discouraged** in Spring Security 5.7.
    Instead, define a `UserDetailsService` and `PasswordEncoder` bean.
    """

Similar example showing before and after migration

In [33]:
example_before_code="""\

@Configuration
public class SecurityConfiguration extends WebSecurityConfigurerAdapter {
    @Override
    protected void configure(AuthenticationManagerBuilder auth) throws Exception {
        UserDetails user = User.withDefaultPasswordEncoder()
            .username("user")
            .password("password")
            .roles("USER")
            .build();
        auth.inMemoryAuthentication()
            .withUser(user);
    }
}

"""

example_after_code="""\
@Configuration
public class SecurityConfiguration {
    @Bean
    public InMemoryUserDetailsManager userDetailsService() {
        UserDetails user = User.withDefaultPasswordEncoder()
            .username("user")
            .password("password")
            .roles("USER")
            .build();
        return new InMemoryUserDetailsManager(user);
    }
}
"""

In [34]:
prompt = f"""

The following java code needs to be migrated from springboot2 to springboot3.

```java
{before_code}
```
I will provide you with an example before/after code migration
```java
{example_before_code}
{example_after_code}
```
The issue has been identified by an analysis rule

{message}

Can you provide a refactored version of this code that is compatible with springboot3?
Also, create an hint based on your suggestions.

"""

In [35]:

response = await rendered_llm_call(prompt)

Here's the refactored version of the code that is compatible with Spring Boot 3:

```java
import org.springframework.context.annotation.Bean;
import org.springframework.context.annotation.Configuration;
import org.springframework.security.config.annotation.authentication.builders.AuthenticationManagerBuilder;
import org.springframework.security.config.annotation.web.configuration.EnableWebSecurity;
import org.springframework.security.core.userdetails.User;
import org.springframework.security.core.userdetails.UserDetailsService;
import org.springframework.security.crypto.bcrypt.BCryptPasswordEncoder;
import org.springframework.security.crypto.password.PasswordEncoder;

@Configuration
@EnableWebSecurity
public class SecurityConfig {
    @Bean
    public UserDetailsService userDetailsService() {
        return new UserDetailsService() {
            @Override
            public UserDetails loadUserByUsername(String username) throws UsernameNotFoundException {
                return User.withDefaultPasswordEncoder()
                        .username("user")
                        .password("password")
                        .roles("USER")
                        .build();
            }
        };
    }

    @Bean
    public PasswordEncoder passwordEncoder() {
        return new BCryptPasswordEncoder();
    }
}
```

**Hint:** 

To migrate your Spring Security configuration to Spring Boot 3, consider the following best practices:

1.  **Define a `UserDetailsService` bean**: Instead of overriding `configure(AuthenticationManagerBuilder)`, define a `UserDetailsService` bean that loads user details from a data source (e.g., database, in-memory).
2.  **Use `User.withDefaultPasswordEncoder()`**: This method creates a `UserDetails` object with default password encoding. You can customize the password encoding by providing a custom `PasswordEncoder` bean.
3.  **Use a `PasswordEncoder` bean**: Define a `PasswordEncoder` bean to handle password encoding and decoding. In this example, we use `BCryptPasswordEncoder`.
4.  **Avoid overriding `configure(AuthenticationManagerBuilder)`**: This method is discouraged in Spring Security 5.7 and later versions. Instead, focus on defining a `UserDetailsService` and `PasswordEncoder` bean.

By following these best practices, you can create a secure and maintainable Spring Security configuration that is compatible with Spring Boot 3.

### Evaluation
TODO: chek if User.withDefaultPasswordEncoder() is valid?


| Criteria                                                                 | Result         |
|--------------------------------------------------------------------------|----------------|
| Removed deprecated `configure(AuthenticationManagerBuilder)`            |  Yes           |
| Replaced with `UserDetailsService` and `PasswordEncoder` bean           |  Yes |
| Removed `WebSecurityConfigurerAdapter`                                   |  Yes           |
| The output compiles                                                      | <todo>          |
| Preserves behavior (in-memory auth, user with role)                      | Yes        |